In [17]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 100_000_000  # about 100mb max file size

In [23]:
class FlowAnimator:
    def __init__(self, width=800, height=400, num_flows=4, lines_per_flow=50,
                 line_height=100, extension=200, wave_speed=0.02, wave_amount=0.5):
        
        """Initialize the animator with configurable parameters."""
        
        # Set up basic dimensions and animation parameters
        self.width, self.height = width, height                     # Canvas dimensions
        self.num_flows, self.lines_per_flow = num_flows, lines_per_flow  # Number of flow centers and lines per center
        self.line_height, self.extension = line_height, extension   # Height of flow area and wave extension
        self.wave_speed, self.wave_amount = wave_speed, wave_amount  # Animation speed and wave amplitude

        # Create and configure matplotlib figure
        self.fig, self.ax = plt.subplots(figsize=(7, 5), facecolor='#FFFBF0')  # Background & background color
        # self.ax.set_facecolor('#FFFBF0')  # Set axes background color. Turned off for now
        self.ax.axis('off')               # Hide axes
        self.ax.set_xlim(0, self.width)   # Set x-axis limits
        self.ax.set_ylim(0, self.height)  # Set y-axis limits
        plt.tight_layout(pad=0)           # Remove padding

        # Calculate evenly spaced positions for flow centers
        self.line_positions = [(i + 1) * width / (num_flows + 1) for i in range(num_flows)]
        self.vertical_lines = self.create_vertical_lines()      # Create static vertical lines
        self.horizontal_lines = self.create_horizontal_lines()  # Create animated horizontal lines

    def create_vertical_lines(self):
        """Create vertical static lines at calculated positions."""
        half_height = self.line_height / 2  # Calculate half height for vertical lines
        
        # Create list of vertical lines at each flow center
        return [self.ax.plot([x, x], [self.height / 2 - half_height, self.height / 2 + half_height],
                             'black', linewidth=2)[0] for x in self.line_positions] # Replace 'black' for other colors

    def create_horizontal_lines(self):
        """Create horizontal animated lines for each flow position."""
        
        # Create nested list of line pairs (left and right) for each flow center
        return [[(self.ax.plot([], [], 'k-', linewidth=0.5, alpha=0.5)[0],  # Left line
                  self.ax.plot([], [], 'k-', linewidth=0.5, alpha=0.5)[0])  # Right line
                 for i in range(self.lines_per_flow)]
                for j in self.line_positions]

    def init(self):
        """Animation Init"""
        
        lines = []
        
        # Reset all horizontal lines to empty data
        for flow_lines in self.horizontal_lines:
            for left_line, right_line in flow_lines:
                left_line.set_data([], [])    # Clear left line
                right_line.set_data([], [])   # Clear right line
                lines.extend([left_line, right_line])
                
        return lines + self.vertical_lines  

    def animate(self, frame):
        """Update geometry at each frame"""
        
        # Calculate current time value
        t = frame * self.wave_speed  
        
        lines = []
        
        # Calculate vertical center points
        center_y, half_height = self.height / 2, self.line_height / 2  

        # Update each flow center's lines
        for flow_idx, center_x in enumerate(self.line_positions):
            for i, (left_line, right_line) in enumerate(self.horizontal_lines[flow_idx]):
                
                # Calculate vertical position for current line
                start_y = center_y - half_height + (self.line_height * i / self.lines_per_flow)
                
                # Generate wave data for both sides
                left_data = self._generate_wave(center_x, start_y, t, left=True)  
                right_data = self._generate_wave(center_x, start_y, t, left=False) 
                
                # Update line positions
                left_line.set_data(*left_data)   
                right_line.set_data(*right_data) 
                lines.extend([left_line, right_line])

        return lines + self.vertical_lines 
        
    def _generate_wave(self, center_x, start_y, t, left=True):
        """Generate waves on horizontal lines"""
        
        # Generate x coordinates based on direction
        x_values = np.linspace(center_x - self.extension if left else center_x,
                               center_x if left else center_x + self.extension, 50)
        
        # Calculate distance factors for wave modulation
        distance_factors = abs(center_x - x_values) / self.extension

        # Opposite phase for right side
        phase_shift = np.pi if not left else 0  
        
        # Create complex wave pattern
        oscillation = np.sin(t + distance_factors * 5 + phase_shift) * \
                      np.sin(t * 0.5 + distance_factors * 3)  # Combine two waves
        
        # Decrease amplitude with distance
        amp_mod = 1 - (distance_factors ** 0.5)  

        # Calculate final y positions
        y_values = start_y + oscillation * self.wave_amount * 20 * amp_mod  
        
        return x_values, y_values

In [24]:
def create_animated_flow():
    animator = FlowAnimator(
        width=800,
        height=400,
        num_flows=4,
        lines_per_flow=50,
        line_height=100,
        extension=200,
        wave_speed=0.1,
        wave_amount=0.5
    )
    
    # Create animation
    anim = FuncAnimation(
        animator.fig,
        animator.animate,
        init_func=animator.init,
        frames=93,  # Adjust frames for loop
        interval=40,
        blit=True
    )
    
    html_video = anim.to_html5_video()

    # Wrap on HTML, custom size
    html_content = f'''
    <video width="600" height="600" controls loop>
        <source src="data:video/mp4;base64,{html_video.split(',')[1]}
    </video>
    '''
    
    plt.close()
    return HTML(html_content)

In [25]:
create_animated_flow()
